In [14]:
from pyhive import hive
import pandas as pd

### Эталонное значение

In [15]:
reference_df = pd.read_csv('artists.csv')

/Users/a18648975/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
reference_df.shape

(1466083, 10)

In [17]:
reference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466083 entries, 0 to 1466082
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   mbid              1466083 non-null  object 
 1   artist_mb         1466075 non-null  object 
 2   artist_lastfm     986756 non-null   object 
 3   country_mb        662368 non-null   object 
 4   country_lastfm    211498 non-null   object 
 5   tags_mb           119946 non-null   object 
 6   tags_lastfm       381075 non-null   object 
 7   listeners_lastfm  986760 non-null   float64
 8   scrobbles_lastfm  986760 non-null   float64
 9   ambiguous_artist  1466083 non-null  bool   
dtypes: bool(1), float64(2), object(7)
memory usage: 102.1+ MB


In [18]:
reference_df.head(2)

,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False


### Задание 2a

In [21]:
# open connection
conn = hive.Connection(host='localhost', port=10000)

In [22]:
df = pd.read_sql(
    """
    SELECT artist_lastfm
    FROM artists
    WHERE scrobbles_lastfm IN (SELECT max(scrobbles_lastfm) FROM artists)
    """
    ,
    conn
)

In [23]:
df

,artist_lastfm
0,The Beatles


### Задание 2b

In [24]:
df_tags = pd.read_sql(
    """
    SELECT tag, COUNT(tag) as tag_count
    FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag
    WHERE tag != ''
    GROUP BY tag
    ORDER BY tag_count DESC
    LIMIT 1
    """
    ,
    conn
)

In [25]:
df_tags

,tag,tag_count
0,seen live,81278


### Задание 2c

In [26]:
df_tags = pd.read_sql(
    """
    SELECT tag, COUNT(tag) as tag_count
    FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag
    WHERE tag != ''
    GROUP BY tag
    ORDER BY tag_count DESC
    LIMIT 10
    """
    ,
    conn
)

In [27]:
df_tags

,tag,tag_count
0,seen live,81278
1,rock,64902
2,electronic,58163
3,All,48631
4,under 2000 listeners,48301
5,alternative,42067
6,pop,41557
7,indie,39333
8,experimental,37665
9,female vocalists,33097


In [28]:
df_artists = pd.read_sql(
    """
    SELECT t.tag as tag,  t.artist_lastfm as artist_lastfm, t.scrobbles_lastfm as scrobbles_lastfm
        FROM(
            SELECT tag, artist_lastfm, scrobbles_lastfm, row_number() over ( partition by tag order by scrobbles_lastfm DESC ) as row_num 
            FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag ) t 
        WHERE row_num = 1
    """
    ,
    conn
)

In [33]:
df_artists.tail(5)

,tag,artist_lastfm,scrobbles_lastfm
403923,zxzw,The Yearlings,10511.0
403924,zxzw 2007,Minitel,1437.0
403925,zydeco,Clifton Chenier,250075.0
403926,électro,Omega Male,5679.0
403927,österreich,King Of Japan,3726.0


In [30]:
df_join = pd.read_sql(
    """
    SELECT *
    FROM (
        SELECT tag, COUNT(tag) as tag_count
        FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag
        WHERE tag != ''
        GROUP BY tag
        ORDER BY tag_count DESC
        LIMIT 10) tag10
    JOIN (
        SELECT t.tag as tag,  t.artist_lastfm as artist_lastfm, t.scrobbles_lastfm as scrobbles_lastfm
            FROM(
                SELECT tag, artist_lastfm, scrobbles_lastfm, row_number() over ( partition by tag order by scrobbles_lastfm DESC ) as row_num 
                FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag ) t 
            WHERE row_num = 1) tag_artists
    ON (tag10.tag = tag_artists.tag)    
    """
    ,
    conn
)

In [31]:
df_join

,tag10.tag,tag10.tag_count,tag_artists.tag,tag_artists.artist_lastfm,tag_artists.scrobbles_lastfm
0,All,48631,All,Ariana Grande,106673208.0
1,alternative,42067,alternative,Coldplay,360111840.0
2,electronic,58163,electronic,Radiohead,499548800.0
3,experimental,37665,experimental,Radiohead,499548800.0
4,female vocalists,33097,female vocalists,Lady Gaga,285469632.0
5,indie,39333,indie,Radiohead,499548800.0
6,pop,41557,pop,The Beatles,517126240.0
7,rock,64902,rock,The Beatles,517126240.0
8,seen live,81278,seen live,Radiohead,499548800.0
9,under 2000 listeners,48301,under 2000 listeners,Joyce Manor,10993278.0


### Задание 2d

In [34]:
# Cамые непопулярные артисты в 10 самых популярных тэгах.
# Выводим по рангу=1, a не row_num=1 
# (поскольку scrobbles_lastfm = 0 (не популярный певец) присутствует у многих исполнителей)

In [35]:
df_join = pd.read_sql(
    """
    SELECT *
    FROM (
        SELECT tag, COUNT(tag) as tag_count
        FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag
        WHERE tag != ''
        GROUP BY tag
        ORDER BY tag_count DESC
        LIMIT 10) tag10
    JOIN (
        SELECT t.tag as tag,  t.artist_lastfm as artist_lastfm, t.scrobbles_lastfm as scrobbles_lastfm
            FROM(
                SELECT tag, artist_lastfm, scrobbles_lastfm, rank() over ( partition by tag order by scrobbles_lastfm ASC ) as row_num 
                FROM artists LATERAL VIEW explode(split(tags_lastfm,';')) tagTable AS tag ) t 
            WHERE row_num = 1) tag_artists
    ON (tag10.tag = tag_artists.tag)    
    """
    ,
    conn
)

In [36]:
df_join

,tag10.tag,tag10.tag_count,tag_artists.tag,tag_artists.artist_lastfm,tag_artists.scrobbles_lastfm
0,All,48631,All,Scorpion,0.0
1,All,48631,All,Haifa,0.0
2,All,48631,All,Bud Powell Trio,0.0
3,All,48631,All,Dave Mellilo,0.0
4,All,48631,All,Hoffmaestro,0.0
...,...,...,...,...,...
1059,under 2000 listeners,48301,under 2000 listeners,Judy Buendia,0.0
1060,under 2000 listeners,48301,under 2000 listeners,Avery Sunshine,0.0
1061,under 2000 listeners,48301,under 2000 listeners,Jen Trynin,0.0
1062,under 2000 listeners,48301,under 2000 listeners,Crash Car Burn,0.0
